# CmpE 59H: Assignment 1 
## Aysun Urhan - November 16, 2018

### 1. MATLAB Codes:
- Function to convert aminoacis sequence from integer to character arrays and vice versa

In [28]:
%%file convertSeq.m
function [outSeq] = convertSeq(inSeq, flag)
% To convert a.a. sequence from int to characters and vice versa
seqSize = size(inSeq);

switch lower(flag)
    case 'to int'
        if isempty(inSeq)
            outSeq = uint8([]);
            return
        end        
        unknown = 0;
        map = uint8(...
            [1 21 5 4 7 14 8 9 10 0 12 11 13 3 0 15 6 2 16 17 0 20 18 23 19 22 24 25 0]);
        
        if unknown ~= 0
            map(map == 0) = unknown;
        end        
        inSeq = lower(inSeq)-96; 
                
        % Adjusting
        inSeq = min(max(inSeq, 0), 27);  
        inSeq(inSeq == 0)  = 29;
        inSeq(inSeq == 27) = 29;
        inSeq(inSeq ==-51) = 28;  
        inSeq(inSeq ==-54) = 27;  
        
        outSeq = map(inSeq);        
        outSeq = reshape(outSeq, seqSize);
        
    case 'to char'
        if isempty(inSeq)
            outSeq = '';
            return
        end
        
        inSeq = inSeq(:);
        inSeq = inSeq';
        
        map = '?ARNDCQEGHILKMFPSTWYVBZX*-?';
 
        inSeq(inSeq > 26) = 0;
        inSeq(inSeq < 0) = 0;
        seqLength = length(inSeq);

        outSeq = char(0);
        outSeq(1, seqLength) = char(0);
        
        for i = 1:seqLength
            outSeq(i) = map(double(inSeq(i)) + 1);
        end
        outSeq = reshape(outSeq, seqSize);
end

Created file 'C:\Users\Aysun\Google Drive\PhD\PhD. - 201819 Fall\CmpE59H\Assignments\Assignment 1\convertSeq.m'.


- Needleman-Wunsch algorithm for global alignment, modified to handle affine gaps

In [29]:
%%file NWAlgo.m
function [M, traceBack, score, alignment] = NWAlgo(intseq1, m, intseq2, n, ScoringMatrix, gapOpen, gapExtend)
% Needleman-Wunsch algorithm modified to handle affine gaps

% Directions when moving on a 2D surface of the matrix:
fDiag = 1; % match/mismatch
fUp   = 2; % gaps in X
fLeft = 3; % gaps in Y
numMoves = 3;
seq1 = convertSeq(intseq1, 'to char');
seq2 = convertSeq(intseq2, 'to char');

% Initializing 3D matrix to keep max scores at each state
M = zeros(n+1, m+1, numMoves);
M(:, 1, :) = -inf;
M(1, :, :) = -inf;
M(1, 1, fDiag) = 0;

M(2:end, 1, fUp)   = gapOpen + gapExtend*(0:n - 1)';
M(1, 2:end, fLeft) = gapOpen + gapExtend*(0:m - 1);

% Initializing a 3D pointer to trace back our path
traceBack(n+1, m+1, numMoves) = uint8(0);
traceBack(2:end, 1, fUp)   = 2;  % Up
traceBack(1, 2:end, fLeft) = 3;  % Left

% Main dynamic programming loop
for i = 2:n + 1
    % Loop for moving on the 2D surface of our matrix
    for j = 2:m + 1
        % Ix
        sUpOpen   = M(i - 1, j, fDiag) + gapOpen; % new gap in seq 1 (X)
        sUpExtend = M(i - 1, j, fUp) + gapExtend; % gap extension in X
       
        % Iy
        sLeftOpen   = M(i, j - 1, fDiag) + gapOpen;   % new gap in seq2 (Y)
        sLeftExtend = M(i, j - 1, fLeft) + gapExtend; % gap extension in Y
        
        % M
        sDiag = M(i - 1, j - 1, fDiag);  % cont diag
        sUp   = M(i - 1, j - 1, fUp);    % end gap in X
        sLeft = M(i - 1, j - 1, fLeft);  % end gap in Y

        % Going up
        if sUpOpen > sUpExtend
            bestUp = sUpOpen; 
            dir = 1; % new gap in X
        elseif sUpOpen < sUpExtend
            bestUp = sUpExtend; 
            dir = 2; % extend gap in X
        else 
            bestUp = sUpOpen; 
            dir = 2; % extend gap in X
        end      
        M(i, j, fUp) = bestUp;
        traceBack(i, j, fUp) = dir;

        % Going left
        if sLeftOpen > sLeftExtend
            bestLeft = sLeftOpen; 
            dir = 1; % new gap in Y
        elseif sLeftOpen < sLeftExtend
            bestLeft = sLeftExtend; 
            dir = 3; % extend gap in Y
        else 
            bestLeft = sLeftOpen; 
            dir = 3; % extend gap in Y
        end
        M(i, j, fLeft) = bestLeft;
        traceBack(i, j, fLeft) = dir;

        % Going diagonal
        [a, b] = max([sDiag, sUp, sLeft]);
        dir = b; 
        bestDiag = a + ScoringMatrix(intseq1(j - 1), intseq2(i - 1));
        M(i, j, fDiag) = bestDiag;
        traceBack(i, j, fDiag) = dir;
    end 
end 

[score, pos] = max(M(end, end, :));
score = score(1); pos = pos(1);

% Alignment
i = n + 1; j = m + 1;
count = 1;
while (i > 1) && (j > 1)
   switch pos
       case 1 % Diag
           path(count, :) = [j - 1, i - 1];
           dir = traceBack(i, j, pos); pos = dir;
           j = j - 1;
           i = i - 1;
       case 2 % Up
           path(count, :) = [0, i - 1];
           dir = traceBack(i, j, pos); pos = dir;
           i = i - 1;
       case 3 % Left
           path(count, :) = [j - 1, 0];
           dir = traceBack(i, j, pos); pos = dir;
           j = j - 1;
       case 0
           break
   end
   if (i == 1) && (i < j)
       i = 2;
   elseif (j == 1) && (j < i)
       j = 2;
   end
   [a, b] = find(path == 1);
   checkEnd = ismember([1, 2], b);
   if sum(checkEnd) > 1
       remSeq1 = a(b == 1);
       path(min(size(path, 1), remSeq1(1))+1:end, 1) = 0;
       remSeq2 = a(b == 2);
       path(min(size(path, 1), remSeq2(1))+1:end, 2) = 0;
       break
   end
   count = count + 1;
end

path = flipud(path);
alignment = repmat(('--')', 1, size(path, 1));
alignment(1, path(:, 1) > 0) = seq1;
alignment(2, path(:, 2) > 0) = seq2;

Created file 'C:\Users\Aysun\Google Drive\PhD\PhD. - 201819 Fall\CmpE59H\Assignments\Assignment 1\NWAlgo.m'.


- Smith-Waterman algorithm for local alignment, modified to handle affine gaps

In [30]:
%%file SWAlgo.m
function [M, traceBack, score, alignment] = SWAlgo(intseq1, m, intseq2, n, ScoringMatrix, gapOpen, gapExtend)
% Smith-Waterman algorithm modified to handle affine gaps

% Directions when moving on a 2D surface of the matrix:
fDiag = 1; % match/mismatch
fUp   = 2; % gaps in X
fLeft = 3; % gaps in Y
numMoves = 3;
seq1 = convertSeq(intseq1, 'to char');
seq2 = convertSeq(intseq2, 'to char');

% Initializing a 3D matrix to keep max scores at each state
M = zeros(n+1, m+1, numMoves);

% Initializing a 3D pointer to trace back our path
traceBack(n+1, m+1, numMoves) = uint8(0);

% Main dynamic programming loop
for i = 2:n + 1
    % Loop for moving on the 2D surface of our matrix
    for j = 2:m + 1
        % Ix
        sUpOpen   = M(i - 1, j, fDiag) + gapOpen; % new gap in seq 1 (X)
        sUpExtend = M(i - 1, j, fUp) + gapExtend; % gap extension in X
       
        % Iy
        sLeftOpen   = M(i, j - 1, fDiag) + gapOpen;   % new gap in seq2 (Y)
        sLeftExtend = M(i, j - 1, fLeft) + gapExtend; % gap extension in Y
        
        % M
        sDiag = M(i - 1, j - 1, fDiag);  % cont diag
        sUp   = M(i - 1, j - 1, fUp);    % end gap in X
        sLeft = M(i - 1, j - 1, fLeft);  % end gap in Y

        % Going up
        if sUpOpen > sUpExtend
            bestUp = sUpOpen; 
            dir = 1; % new gap in X
        elseif sUpOpen < sUpExtend
            bestUp = sUpExtend; 
            dir = 2; % extend gap in X
        else 
            bestUp = sUpOpen; 
            dir = 2; % extend gap in X
        end   
        if bestUp > 0
            M(i, j, fUp) = bestUp;
            traceBack(i, j, fUp) = dir;
        else
            M(i, j, fUp) = 0;
            traceBack(i, j, fUp) = 0;
        end
        
        % Going left
        if sLeftOpen > sLeftExtend
            bestLeft = sLeftOpen; 
            dir = 1; % new gap in Y
        elseif sLeftOpen < sLeftExtend
            bestLeft = sLeftExtend; 
            dir = 3; % extend gap in Y
        else 
            bestLeft = sLeftOpen; 
            dir = 3; % extend gap in Y
        end
        if bestLeft > 0
            M(i, j, fLeft) = bestLeft;
            traceBack(i, j, fLeft) = dir;
        else
            M(i, j, fLeft) = 0;
            traceBack(i, j, fLeft) = 0;
        end

        % Going diagonal
        [a, b] = max([sDiag, sUp, sLeft]);
        dir = b; 
        bestDiag = a + ScoringMatrix(intseq1(j - 1), intseq2(i - 1));
        if bestDiag > 0
            M(i, j, fDiag) = bestDiag;
            traceBack(i, j, fDiag) = dir;
        else
            M(i, j, fDiag) = 0;
            traceBack(i, j, fDiag) = 0;
        end
    end 
end 

score   = max(M(:));
[i, j, pos] = ind2sub(size(M), find(M == score, 1));

% Alignment
count = 1;
while (i > 1) && (j > 1)
    switch pos
        case 1 % Diag
            path(count, :) = [j - 1, i - 1];
            dir = traceBack(i, j, pos); pos = dir;
            j = j - 1;
            i = i - 1;
        case 2 % Up
            path(count, :) = [0, i - 1];
            dir = traceBack(i, j, pos); pos = dir;
            i = i - 1;
        case 3 % Left
            path(count, :) = [j - 1, 0];
            dir = traceBack(i, j, pos); pos = dir;
            j = j - 1;
        case 0
            break
    end
    if (i == 1) && (i < j)
        i = 2;
    elseif (j == 1) && (j < i)
        j = 2;
    end
    [a, b] = find(path == 1);
    checkEnd = ismember([1, 2], b);
    if sum(checkEnd) > 1
        remSeq1 = a(b == 1);
        path(min(size(path, 1), remSeq1(1))+1:end, 1) = 0;
        remSeq2 = a(b == 2);
        path(min(size(path, 1), remSeq2(1))+1:end, 2) = 0;
        break
    end
    count = count + 1;
end

if (count > 1) && (count < max(m, n))
    path(end, :) = [];
end
path = flipud(path);

alignment = repmat(('--')', 1, size(path, 1));
alignment(1, path(:, 1) > 0) = seq1(path(path(:, 1) > 0, 1));
alignment(2, path(:, 2) > 0) = seq2(path(path(:, 2) > 0, 2));

Created file 'C:\Users\Aysun\Google Drive\PhD\PhD. - 201819 Fall\CmpE59H\Assignments\Assignment 1\SWAlgo.m'.


- Main function for alignment, it calls the subfunctions above

In [31]:
%%file alignSeq.m
function [score, alignment, M, traceBack] = alignSeq(seq1, seq2, gapOpen, gapExt, flagA)

% Read the scoring matrix
tmp = importdata('BLOSUM62.txt', ' ', 7);
ScoringMatrix = tmp.data;

% Just to make sure the gap penalties are negative
if sign(gapOpen) > 0
    gapOpen = -1*gapOpen;
end
if sign(gapExt) > 0
    gapExt  = -1*gapExt;
end

% Converting the aa sequence to int array if it's a character array
if ischar(seq1)
    seq1    = upper(seq1); % the output alignment will be all uppercase
    intseq1 = convertSeq(seq1, 'to int');
else
    intseq1 = uint8(seq1);
    seq1    = convertSeq(intseq1, 'to char');
end
if ischar(seq2)
    seq2    = upper(seq2); % the output alignment will be all uppercase
    intseq2 = convertSeq(seq2, 'to int');
else
    intseq2 = uint8(seq2);
    seq2    = convertSeq(intseq2, 'to char');
end

m = length(seq1);
n = length(seq2);

if ~n||~m
    error('Empty strings!!');
end

% To handle unknown/weird characters outside our scoring matrix
% map all foreign objects to 'X'
weirdo      = convertSeq('X', 'to int');
maxAllowed  = size(ScoringMatrix, 1);
highestVal  = max([intseq1, intseq2]);
if highestVal > maxAllowed
    % if the matrix contains the 'Any' we map to that
    intseq1(intseq1 > maxAllowed) = weirdo;
    intseq2(intseq2 > maxAllowed) = weirdo;
end

switch lower(flagA)
    case 'global'
        [M, traceBack, score, alignment] = NWAlgo(intseq1, m, intseq2, n, ...
            ScoringMatrix, gapOpen, gapExt);
    case 'local'
        [M, traceBack, score, alignment] = SWAlgo(intseq1, m, intseq2, n, ...
            ScoringMatrix, gapOpen, gapExt);
    otherwise
        error('Specify your algorithm!!');
end

Created file 'C:\Users\Aysun\Google Drive\PhD\PhD. - 201819 Fall\CmpE59H\Assignments\Assignment 1\alignSeq.m'.


### 2. Example Use with Test Pairs
#### Global alignment

- Test pair 1:

In [32]:
testPair = importdata('test-pair1.txt'); % reading sequences
seq1 = testPair{1};
seq2 = testPair{2};
gapOpen = 11; % gap opening penalty
gapExt  = 1;  % gap extension penalty
algo    = 'global'; % specify type of aligment
[score, alignment] = alignSeq(seq1, seq2, gapOpen, gapExt, algo);
score, alignment


score =

    -1


alignment =

  2x10 char array

    'PLEASANTLY'
    '-MEAN---LY'




- Test pair 2:

In [33]:
testPair = importdata('test-pair2.txt'); % reading sequences
seq1 = testPair{1};
seq2 = testPair{2};
gapOpen = 11; % gap opening penalty
gapExt  = 1;  % gap extension penalty
algo    = 'global'; % specify type of aligment
[score, alignment] = alignSeq(seq1, seq2, gapOpen, gapExt, algo);
score, alignment


score =

     8


alignment =

  2x10 char array

    'PRT---EINS'
    'PRTWPSEIN-'




- Test pair 3:

In [34]:
testPair = importdata('test-pair3.txt'); % reading sequences
seq1 = testPair{1};
seq2 = testPair{2};
gapOpen = 11; % gap opening penalty
gapExt  = 1;  % gap extension penalty
algo    = 'global'; % specify type of aligment
[score, alignment] = alignSeq(seq1, seq2, gapOpen, gapExt, algo);
score, alignment


score =

   144


alignment =

  2x92 char array

    'YHFDVPDCWAHRYWVENPQAIAQME-------QICFNWFPSMMMK-------QPHVF---KVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE'
    'YHEDV----AHE------DAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPI----ISATCARMRVRTVWE'




#### Local alignment
- Test pair 1:

In [35]:
testPair = importdata('test-pair1.txt'); % reading sequences
seq1 = testPair{1};
seq2 = testPair{2};
gapOpen = 11; % gap opening penalty
gapExt  = 1;  % gap extension penalty
algo    = 'local'; % specify type of aligment
[score, alignment] = alignSeq(seq1, seq2, gapOpen, gapExt, algo);
score, alignment


score =

    12


alignment =

  2x4 char array

    'LEAS'
    'MEAN'




- Test pair 2:

In [36]:
testPair = importdata('test-pair2.txt'); % reading sequences
seq1 = testPair{1};
seq2 = testPair{2};
gapOpen = 11; % gap opening penalty
gapExt  = 1;  % gap extension penalty
algo    = 'local'; % specify type of aligment
[score, alignment] = alignSeq(seq1, seq2, gapOpen, gapExt, algo);
score, alignment


score =

    19


alignment =

  2x9 char array

    'PRT---EIN'
    'PRTWPSEIN'




- Test pair 3:

In [37]:
% Test pair 3:
testPair = importdata('test-pair3.txt'); % reading sequences
seq1 = testPair{1};
seq2 = testPair{2};
gapOpen = 11; % gap opening penalty
gapExt  = 1;  % gap extension penalty
algo    = 'local'; % specify type of aligment
[score, alignment] = alignSeq(seq1, seq2, gapOpen, gapExt, algo);
score, alignment


score =

   144


alignment =

  2x92 char array

    'YHFDVPDCWAHRYWVENPQAIAQME-------QICFNWFPSMMMK-------QPHVF---KVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE'
    'YHEDV----AHE------DAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPI----ISATCARMRVRTVWE'


